# VLSP synonyms

Issue 1: Variations in capitalised and uncapitalised letters. 
Solution: Take the headword as the canonical spelling.

Issue 2: Missing synonym annotation for a headword, even though the said headword is annotated as a synonym for another headword.
Partial solution: Remove subsets.

In [1]:
import os
import random

import pandas as pd

from typing import Optional, List, Dict

In [2]:
df = pd.read_csv("../data/VietMDR_VLSP_SP72/vlsp.csv") 
df

,headword,pos,definition,synonyms,antonyms
0,a,N,con chữ thứ nhất của bảng chữ cái chữ quốc ngữ,NaN,NaN
1,a,O,"tiếng thốt ra biểu lộ sự vui mừng, ngạc nhiên ...",NaN,NaN
2,a,N,are [viết tắt],NaN,NaN
3,A,N,ampere [viết tắt],NaN,NaN
4,A,N,"kí hiệu phân loại trên dưới, thứ nhất [trước B]",NaN,NaN
...,...,...,...,...,...
41729,z,N,"[đọc là ""dét""] con chữ của bảng chữ cái Latin,...",NaN,NaN
41730,zero,N,số không,NaN,NaN
41731,zeta,N,"con chữ thứ sáu của bảng chữ cái Hi Lạp [ζ, Z]",NaN,NaN
41732,zigzag,N,xem <i>dích dắc</i>,dích dắc,NaN


In [3]:
df_syns = df[~df["synonyms"].isna()]
df_syns

,headword,pos,definition,synonyms,antonyms
6,a dua,V,bắt chước theo người khác làm việc sai trái của,"a tòng, vào hùa",NaN
9,a ma tơ,A,"[phong cách, lối làm việc] phóng túng, tuỳ thí...",tài tử,NaN
11,a tòng,V,tham gia vào [việc làm sai trái dưới sự điều k...,"a dua, vào hùa",NaN
14,à ơi,O,tiếng đệm trong lời ru,"ạ ơi, ầu ơ",NaN
16,ả,N,từ dùng để chỉ người phụ nữ nào đó với ý coi t...,thị,NaN
...,...,...,...,...,...
41705,yêu quý,V,yêu mến và quý trọng,yêu quí,NaN
41708,yêu thương,V,có tình cảm gắn bó tha thiết và quan tâm chăm ...,thương yêu,NaN
41724,yếu tố,N,"bộ phận cấu thành một sự vật, sự việc, hiện tượng",nguyên tố,NaN
41725,yếu tố,N,cái cần thiết tạo điều kiện hình thành nên cái...,"nguyên tố, nhân tố",NaN


In [4]:
def get_syn_grp(row: pd.Series):
  syn_text: str = row["synonyms"]

  while True:
    i = syn_text.find('[')
    if i < 0:
      break
    j = syn_text.find(']', i)
    if j == -1:
      raise Exception(f"Missing closing square bracket in {syn_text}")
    syn_text = syn_text[:i] + syn_text[j + 1:]
      
  lst = [x.strip() for x in syn_text.split(',')]
  lst = [x for x in lst if x]
  lst.append(row["headword"])
  
  lst = list(set(lst))
  lst.sort()
  
  return tuple(set(lst))
  
syn_grps = set(df_syns.apply(get_syn_grp, axis=1))

print(len(syn_grps))
print(random.choices([*syn_grps],k=20))

7401
[('rặt', 'toàn', 'tinh', 'tuyền'), ('lầm', 'lẫn', 'lộn'), ('cát kết', 'sa thạch'), ('bí đao', 'bí phấn'), ('ruột rà', 'ruột thịt'), ('tiền liệt tuyến', 'tuyến tiền liệt'), ('ứng', 'tương ứng'), ('cao nghệu', 'cao nghều'), ('mất trộm', 'mất cắp'), ('chè', 'trà'), ('bằng', 'kì'), ('phiên dịch', 'biên dịch'), ('phúc tinh', 'cứu tinh'), ('không nhiều thì ít', 'không ít thì nhiều'), ('ngờ vực', 'nghi hoặc'), ('nhị cái', 'nhuỵ'), ('sẽ sàng', 'khẽ khàng'), ('tàn', 'rụi', 'lụi', 'tàn lụi'), ('tròng đen', 'lòng đen'), ('ti thể', 'thể sợi')]


## Remove subsets from groups of synonyms

In [5]:
inverted_idx: Dict[str, List[tuple]] = dict()

for grp in syn_grps:
  for word in grp:
    if word not in inverted_idx:
      inverted_idx[word] = [grp]
    else:
      inverted_idx[word].append(grp)

for grps in inverted_idx.values():
  grp_sets = [set(grp) for grp in grps]
  for i, grp_i in enumerate(grp_sets):
    if grps[i] not in syn_grps:
      continue
    for j, grp_j in enumerate(grp_sets):
      if grp_j is grp_i:
        continue
      if grp_i.issubset(grp_j):
        syn_grps.remove(grps[i])
        break

In [6]:
syn_grps = list(syn_grps)

syn_grps.sort(key=lambda grp: tuple(word.lower() for word in grp))
print(len(syn_grps))
print(syn_grps[:20])

6441
[('a ma tơ', 'tài tử'), ('a tòng', 'vào hùa', 'a dua'), ('accordeon', 'đàn xếp', 'phong cầm'), ('AIDS', 'SIDA'), ('am tường', 'thông thuộc', 'am hiểu', 'thông tỏ', 'thông hiểu'), ('amygdala', 'hạnh nhân'), ('an toạ', 'yên vị', 'an vị'), ('an táng', 'chôn cất', 'mai táng'), ('an táng', 'mai táng', 'táng'), ('anh hùng ca', 'sử thi'), ('ao ước', 'ước ao'), ('azot', 'nitrogen'), ('ba que', 'đểu cáng', 'xỏ lá', 'đểu giả'), ('ba rọi', 'ba chỉ'), ('ban sơ', 'ban đầu'), ('banh', 'bóng'), ('banh', 'chành', 'nhành'), ('banh', 'chành', 'vành'), ('bao biện', 'bào chữa', 'biện hộ'), ('bao biện', 'ôm đồm')]


In [7]:
with open("../data/VietMDR_VLSP_SP72/syns.txt", 'w') as f:
  for grp in syn_grps:
    f.write(','.join(grp))
    f.write('\n')